In [1]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification,AdamW, BertConfig, BertModel, BertPreTrainedModel
import pytorch_lightning as pl
#MODEL_NAME='bert-base-cased'
from torch import nn
import MeCab
batch=2
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tagger = MeCab.Tagger()
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [2]:
from sklearn.preprocessing import LabelEncoder


In [3]:
df = pd.read_csv('/data1/ohnishi/202271month_per_hour_geotaged_adGeocode_undersampled.csv')
UserTweets = pd.read_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/noGeo/202271month_per_hour_noGeo.csv',engine='python')

In [4]:
y = "code"
L = LabelEncoder()
df[y] = L.fit_transform(df[y])
num_class = df[y].max() + 1  # オリジンの補正

In [5]:
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers import BertModel
class BertForSequenceClassifier_pl(pl.LightningModule):
        
    def __init__(self, model_name, lr, num_class):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_class)
        self.criterion = nn.CrossEntropyLoss()

        # BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        preds= self.classifier(output.pooler_output)
        
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        #print(f"tihi is {loss}")
        return loss, preds
    
       # trainのミニバッチに対して行う処理
    def training_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        self.log('train_loss', loss)
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # validation、testでもtrain_stepと同じ処理を行う
    def validation_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    def test_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # epoch終了時にvalidationのlossとaccuracyを記録
    def validation_epoch_end(self, outputs, mode="val"):
        # loss計算
        epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f"{mode}_loss", epoch_loss, logger=True)

        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"{mode}_accuracy", epoch_accuracy, logger=True)

        

    # testデータのlossとaccuracyを算出（validationの使いまわし）
    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, "test")

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [6]:
best_model_path='/home/is/shuntaro-o/dev/persons_move_analysis/models/japan_model_ver2.ckpt'

In [7]:
x = "text"
UserTweets[x] = UserTweets[x].astype(str)
UserTweets[x]=UserTweets[x].astype(str)
UserTweets[x] = UserTweets[x].str.replace(r'[【】]', ' ')       # 【】の除去
UserTweets[x] = UserTweets[x].str.replace(r'[（）()]', ' ')     # （）の除去
UserTweets[x] = UserTweets[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
UserTweets[x] = UserTweets[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
UserTweets[x]= UserTweets[x].str.replace(r'_', '')#underscoUserTweets[x].str 
UserTweets[x] = UserTweets[x].str.replace(r'https?:\/\/.*?[\r\n ]', '')  # URLの除去
UserTweets[x] = UserTweets[x].str.replace(r'http:\/\/.*?[\r\n ]', '')  # URLの除去
UserTweets[x] = UserTweets[x].str.replace(r'　', ' ')

/tmp/ipykernel_5599/2855484960.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  UserTweets[x] = UserTweets[x].str.replace(r'[【】]', ' ')       # 【】の除去
/tmp/ipykernel_5599/2855484960.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  UserTweets[x] = UserTweets[x].str.replace(r'[（）()]', ' ')     # （）の除去
/tmp/ipykernel_5599/2855484960.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  UserTweets[x] = UserTweets[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
/tmp/ipykernel_5599/2855484960.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  UserTweets[x] = UserTweets[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
/tmp/ipykernel_5599/2855484960.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  UserTweets[x] = UserTweets[x].str.replace(r'https?:\/\/

In [8]:
sentences_text_test=UserTweets.text.values
list_estimate = []
model = BertForSequenceClassifier_pl.load_from_checkpoint(best_model_path)
bert=model.bert.cuda()
classifier=model.classifier.cuda()
for sentence in sentences_text_test:
    encoding = tokenizer(
    sentence,
    padding = 'longest',
    return_tensors='pt')
    encoding = { k: v.cuda() for k, v in encoding.items() }
    with torch.no_grad():
        output = bert(**encoding)
        ans=classifier(output.pooler_output)
        ans = ans.to('cpu').detach().numpy().copy()
        ans = ans.argmax()
        ans = L.inverse_transform([ans])[0]
        list_estimate.append(ans)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
list_estimate=pd.Series(list_estimate)
noGeotweets=pd.concat([UserTweets,list_estimate],axis=1)
noGeotweets=noGeotweets.rename(columns={0: 'code'})

In [14]:
noGeotweets.to_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/202271month_per_hour_noGeo_adEstimate.csv')

In [18]:
noGeotweets

,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,author_id,username,author_followers,author_tweets,author_description,author_location,code
0,0,帯だぞおおお＼ 'ω' ／ウオオオ.ᐟ.ᐟ.ᐟ.ᐟ女の子の日に負けずに頑張る ง •̀•́...,2022-07-01_00:59:58,2.0,0.0,11.0,0.0,1423308910274834435,m_i_k_u_chun,319.0,5620,8/3 start✨10.11.1.5.6.9月公式パートナー👑シマエナガ大好き💕［定時］毎...,NaN,5439
1,1,RT : ／#原神 Ver.2.7アップデート記念キャンペーン！第3弾3日目＼7月4日 月 ...,2022-07-01_00:59:58,92365.0,0.0,0.0,0.0,1009827276182056961,musubi_kou,500.0,10912,男型の人間体、女形の鉱石体を持つ半分人間半分結晶のVtuber 結ノ鉱(むすびのこう)と申し...,何処かの洞窟の奥深く,5239
2,2,モンサントの利権のためかも,2022-07-01_00:59:58,1.0,1.0,9.0,0.0,134441975,yonedaixtuki,521.0,9941,趣味はプラモデル作りです。好きなものは、アリスギアアイギス、サクラ大戦、うたわれるもの、ガン...,NaN,5030
3,3,RT : &gt; アダルト産業を実質的に規制しているのは政府や国際条約ではなく「クレジット...,2022-07-01_00:59:58,1961.0,0.0,0.0,0.0,57352324,sakenotori,528.0,38960,＊おおっと＊,＊いしのなかにいる＊,5135
4,4,私が様々な知識を紹介しているのは、もちろんキャンプを楽しんでほしいという気持ちもありますが、...,2022-07-01_00:59:58,0.0,0.0,49.0,0.0,1362723804347310081,Mas_CampingUK,3896.0,8565,イギリス在住の日系英国人｜キャンプ歴40年｜タープシェルター（#MAS1／#MAS1W／#M...,Northern England,5339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357558,460,夜勤中の仮眠時間中😪自分へのご褒美のつもりで全公演2Days応募した！当たりますように！！！！！🙏,2022-07-31_23:59:58,0.0,1.0,3.0,0.0,909037113135439873,noi_LP,82.0,4927,L&P7年目20↑/うたプリトキヤ音也蘭丸/宮野真守/鈴木達央/いろいろマイペースに楽しんで...,Next→ENTERTAINING!全公演,5235
357559,461,RT : 眩しすぎて泣けてくる、ｱｲﾄﾞﾙになる為に生まれてきた子 https://t.co...,2022-07-31_23:59:58,250.0,0.0,0.0,0.0,1427417938181582853,g1PNCHcDzxmLpXE,127.0,15967,NaN,NaN,5339
357560,462,RT : COVID-19陽性患者のオペをfull PPEでやって消耗した。COVID-19...,2022-07-31_23:59:58,266.0,0.0,0.0,0.0,1175984276728205313,q4VB58cKuT4YGYO,4.0,332,NaN,NaN,5235
357561,463,フランす‼️＃CountryHumans ＃カントリーヒューマンズ https://t.co...,2022-07-31_23:59:58,7.0,2.0,77.0,0.0,1501204461762801665,mtpmm_nagase,587.0,6841,⚠️旧国 政治的意図は無いです。,アイコン→社畜さん‼︎ ヘッダー→あまや あるふぁとの合作,5339


In [19]:
df_Tokyo = noGeotweets[noGeotweets['code']==5339]

In [20]:
df_Tokyo.to_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/202271month_per_hour_noGeo_adEstimate_Tokyo.csv')